In [1]:
cd drive/MyDrive/OnlineCourse/WorldModel/WorldModels

/content/drive/MyDrive/OnlineCourse/WorldModel/WorldModels


In [2]:
!pip install -r ./requirements/gym.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616822 sha256=8469845b013514fbd70e5515a71a075bb23324ef0b764387b666015978b562c7
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [3]:
!python src/tools/train/single_env_train.py

pybullet build time: May 20 2022 19:44:17
tcmalloc: large alloc 2457600000 bytes == 0x4bd42000 @  0x7fbe5ae4c680 0x7fbe5ae6c2ec 0x7fbe5999022f 0x7fbe599e868b 0x7fbe599e94f7 0x7fbe59a8b913 0x5c4677 0x56cc92 0x569d8a 0x5f60c3 0x59c130 0x5f666f 0x57165d 0x569d8a 0x68e267 0x67d9b1 0x67da2f 0x67dad1 0x67fbf7 0x6b8082 0x6b840d 0x7fbe5ac4b083 0x5faa2e
argv[0]=
argv[0]=
argv[0]=
argv[0]=
episode [  11/  50] is collected. Total reward is -1846.870343
elasped time for interaction: 9.19s
update_step:   1 model loss: 1257.77112, kl_loss: 3.00000, obs_loss: 1249.78247, reward_loss: 4.98862, value_loss: 0.06165 action_loss: 0.27860
update_step:   2 model loss: 1246.25305, kl_loss: 3.00000, obs_loss: 1238.90369, reward_loss: 4.34931, value_loss: 0.28659 action_loss: 0.66787
update_step:   3 model loss: 1217.67688, kl_loss: 3.00000, obs_loss: 1209.85498, reward_loss: 4.82195, value_loss: 0.74478 action_loss: 1.10951
update_step:   4 model loss: 1187.90759, kl_loss: 3.00000, obs_loss: 1180.77930, rewar

In [2]:
from src.model import ActionModel, Encoder, RSSM, ValueModel

In [3]:
rssm = RSSM(30, 8, 200,  'cpu')
value_model = ValueModel(30, 200)
action_model = ActionModel(30, 200, 8)
encoder = Encoder()

In [29]:
import os
import torch
import torch.nn as nn

def get_fractional(saved_state, weight_key:str, ratio: float):
    fractional = saved_state[weight_key] * ratio
    return fractional

log_dir = './logs/train_[AntBulletEnv-v0_HopperBulletEnv-v0]_test_[HalfCheetahBulletEnv-v0]_20230127-155422/episode_0020'

encoder.load_state_dict(torch.load(os.path.join(log_dir, "encoder.pth")))

action_model.load_state_dict(torch.load(os.path.join(log_dir, "action_model.pth")))
action_model.fc4.weight = nn.Parameter(torch.Tensor(action_model.fc4.weight.size()))
action_model.fc4.bias = nn.Parameter(torch.Tensor(action_model.fc4.bias.size()))

value_fc4_init_weight = value_model.fc4.weight
value_fc4_init_bias = value_model.fc4.bias
value_model.load_state_dict(torch.load(os.path.join(log_dir, "value_model.pth")))
value_model.fc4.weight = nn.Parameter(value_fc4_init_weight + get_fractional(value_model.state_dict(), 'fc4.weight', 0.2))
value_model.fc4.bias = nn.Parameter(value_fc4_init_bias + get_fractional(value_model.state_dict(), 'fc4.bias', 0.2))

reward_fc4_init_weight = rssm.reward.fc4.weight
reward_fc4_init_bias = rssm.reward.fc4.bias
rssm.reward.load_state_dict(torch.load(os.path.join(log_dir, "reward_model.pth")))
rssm.reward.fc4.weight = nn.Parameter(reward_fc4_init_weight + get_fractional(rssm.reward.state_dict(), 'fc4.weight', 0.2))
rssm.reward.fc4.bias = nn.Parameter(reward_fc4_init_bias + get_fractional(rssm.reward.state_dict(), 'fc4.bias', 0.2))



In [30]:
reward_fc4_init_weight = rssm.reward.fc4.weight
reward_fc4_init_bias = rssm.reward.fc4.bias
rssm.reward.load_state_dict(torch.load(os.path.join(log_dir, "reward_model.pth")))
rssm.reward.fc4.weight = nn.Parameter(reward_fc4_init_weight + get_fractional(rssm.reward.state_dict(), 'fc4.weight', 0.2))
rssm.reward.fc4.bias = nn.Parameter(reward_fc4_init_bias + get_fractional(rssm.reward.state_dict(), 'fc4.bias', 0.2))



In [25]:
get_fractional(value_model.state_dict(), 'fc4.weight', 0.2)

tensor([[-1.4916e-03, -6.3031e-03,  9.5848e-03,  5.1033e-03,  5.2286e-03,
         -3.6344e-03, -8.7178e-04, -8.6799e-03, -9.8946e-03, -1.3835e-03,
          9.4870e-03, -6.0676e-03,  4.0218e-04, -9.2144e-03,  3.9033e-04,
          7.2913e-03, -5.3362e-03, -6.8697e-03, -1.2719e-03, -8.8303e-03,
          1.9492e-03, -5.5750e-03,  2.3939e-03, -7.8412e-03, -8.8629e-03,
         -3.0062e-03, -3.5252e-03,  3.3299e-03,  8.9445e-04,  3.5547e-03,
          6.6692e-03,  5.9819e-03,  7.9971e-04,  4.7064e-03, -6.7325e-03,
         -8.0008e-03,  3.2633e-03, -4.2876e-03, -9.5823e-03,  2.5990e-03,
         -2.4897e-04, -1.0529e-03,  5.7503e-03, -9.9288e-04,  3.6259e-03,
         -3.9959e-03,  6.2206e-03,  9.2169e-03,  1.7157e-03,  4.6766e-03,
         -5.9347e-03, -4.1814e-03, -3.0639e-03,  1.7112e-03,  6.9981e-03,
         -9.8481e-03, -4.5471e-03, -4.7596e-03,  3.6638e-03,  7.8313e-03,
          5.3356e-03,  6.0130e-03, -7.6074e-03,  1.5089e-03,  8.5936e-03,
          4.5296e-03, -9.2204e-03, -1.

In [24]:
value_model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.0498, -0.0044,  0.0326,  ..., -0.0464, -0.0285,  0.0192],
                      [-0.0658, -0.0371, -0.0236,  ..., -0.0255, -0.0520,  0.0247],
                      [-0.0433,  0.0005,  0.0028,  ..., -0.0255,  0.0612, -0.0008],
                      ...,
                      [-0.0557,  0.0492, -0.0176,  ...,  0.0059, -0.0645, -0.0537],
                      [ 0.0570, -0.0007, -0.0322,  ...,  0.0021,  0.0224, -0.0358],
                      [ 0.0363,  0.0481,  0.0152,  ...,  0.0150,  0.0159, -0.0177]])),
             ('fc1.bias',
              tensor([ 7.7319e-03, -3.5356e-02, -1.0222e-02,  7.2797e-04, -9.2736e-03,
                      -1.9465e-02,  1.1184e-03,  2.2447e-02,  4.1885e-03, -5.6111e-02,
                       2.5874e-02,  6.0444e-02, -4.6454e-02, -2.2138e-03, -1.2065e-03,
                       4.2896e-02,  7.7669e-03, -5.1671e-02, -5.3808e-02,  4.0790e-03,
                       1.2436e-02,  1.4327e-02, -2.4535e-04,  1.

In [36]:
rssm = RSSM(30, 8, 200,  'cpu')

saved_state = rssm.reward.state_dict()
weight_key = 'fc4.weight'
ratio = 0.2

def get_fractional(saved_state, weight_key:str, ratio: float):
    fractional = saved_state[weight_key] * ratio
    return fractional

get_fractional(saved_state, weight_key, ratio)

tensor([[-8.9684e-03,  5.7014e-03, -4.4616e-03,  9.4157e-03, -7.2291e-03,
         -6.6591e-03, -7.9339e-03, -1.2123e-03,  4.7958e-03, -2.5051e-03,
          6.1084e-03, -8.6322e-03,  5.9419e-03, -6.7320e-03, -3.6118e-04,
          7.4822e-04,  7.3255e-03,  4.4139e-03, -6.0288e-03, -9.7699e-03,
          2.5142e-03, -5.2730e-03, -3.0453e-04, -3.0926e-03,  3.1187e-03,
          8.8140e-04, -4.6294e-03,  7.1031e-03, -7.8501e-03, -4.4708e-03,
          4.0547e-03, -5.5584e-03,  4.7583e-03,  3.9500e-03,  5.5370e-03,
         -2.2364e-03, -5.1672e-03, -3.0474e-03, -2.6281e-03,  2.8932e-03,
         -1.7277e-03,  4.9655e-03,  3.1357e-03, -6.7384e-03,  7.1649e-03,
          5.4864e-03, -6.4155e-03, -2.6448e-03,  4.6894e-04,  1.4788e-03,
          1.1715e-04,  9.7172e-03, -8.7966e-03,  5.0566e-03,  1.2627e-03,
          6.8853e-03, -5.0849e-03,  4.9428e-03,  3.0661e-03,  8.9277e-03,
          7.7597e-03,  3.6724e-03,  2.1534e-03, -3.4066e-03, -2.1701e-03,
          7.1023e-03, -8.7315e-03,  3.

In [15]:
torch.Tensor(4)

tensor([8.3225e+08, 0.0000e+00, 3.7835e-44, 0.0000e+00])

In [25]:
state = rssm.reward.state_dict()['fc4.weight'] * 0.2

In [29]:
import copy
init = copy.deepcopy(rssm.reward.fc4.weight)
after = rssm.reward.fc4.weight + state

In [33]:
import torch
torch.all(after == init)

tensor(False)